In [ ]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline

In [ ]:
np.random.seed(42)

# Portfolio Planner

In this activity, you will use the Alpaca api to grab historical data for a 60/40 portfolio using `SPY` to represent the stock portion and `AGG` to represent the bonds.

In [ ]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = "PKQ0EQ43YZ8AQG0TEX3B"#os.getenv("ALPACA_API_KEY")
alpaca_secret_key = "N9PUxnUxw9m7D6/E0Oreh3TUHcMkPpfNA8n5SHZO"#os.getenv("ALPACA_SECRET_KEY")

# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

# Data Collection

In this step, you will need to use the Alpaca api to fetch closing prices for the `SPY` and `AGG` tickers. Save the results as a pandas DataFrame

In [ ]:
list_of_tickers = ["SPY", "AGG"]
barset = api.get_barset(list_of_tickers, 'day', limit = 252)
spy_bars = barset['SPY']
spy_close = []
for bar in spy_bars:
    spy_close.append(bar.c)
agg_bars = barset['AGG']
agg_close = []
for bar in agg_bars:
    agg_close.append(bar.c)
df = pd.DataFrame({'SPY': spy_close, 'AGG': agg_close})
df

# Monte Carlo Simulation

In this step, you will run Monte Carlo Simulations for your portfolio to model portfolio performance at different retirement ages. 

Complete the following steps:
1. Calculate the daily returns for the SPY and AGG closing prices.
2. Calculate volatility for both the SPY and AGG closing prices.
3. Find the last day's closing price for both stocks and save those as variables.
4. Run a Monte Carlo Simulation of at least 500 iterations and generate at least 30 years of closing prices

### HINTS:
There are 252 trading days per year, so the number of records to generate for each Monte Carlo run will be 252 days * 30 years

In [ ]:
# Calculate the daily roi for the stocks
daily_returns = df.pct_change()
daily_returns.head()

In [ ]:
spy_avg_daily_return = daily_returns.mean()['SPY']
spy_avg_daily_return

In [ ]:
agg_avg_daily_return = daily_returns.mean()['AGG']
agg_avg_daily_return

In [ ]:
# Calculate volatility
spy_std_dev_daily_return = daily_returns.std()['SPY']
spy_std_dev_daily_return

In [ ]:
agg_std_dev_daily_return = daily_returns.std()['AGG']
agg_std_dev_daily_return

In [ ]:
# Save the last day's closing price
spy_last_price = df['SPY'].iloc[-1]
agg_last_price = df['AGG'].iloc[-1]

In [ ]:
simulated_spy_prices = [spy_last_price]
simulated_agg_prices = [agg_last_price]

In [ ]:
# Setup the Monte Carlo Parameters
number_simulations = 500
number_records = 252 * 30
monte_carlo = pd.DataFrame()
weights = [0.60, 0.40]

In [ ]:
# Run the Monte Carlo Simulation
for x in range(number_simulations):
    for i in range(number_records):
        simulated_spy_price = simulated_spy_prices[-1] * (1 + np.random.normal(spy_avg_daily_return, spy_std_dev_daily_return))
        simulated_agg_price = simulated_agg_prices[-1] * (1 + np.random.normal(agg_avg_daily_return, agg_std_dev_daily_return))
   
        simulated_spy_prices.append(simulated_spy_price)
        simulated_agg_prices.append(simulated_agg_price)
        
    monte_carlo["SPY Prices"] = simulated_spy_prices
    monte_carlo["AGG Prices"] = simulated_agg_prices
    
    simulated_daily_returns = monte_carlo.pct_change()
    
    portfolio_cumulative_returns = simulated_cumulative_returns.dot(weights)
    
    simulated_cumulative_returns[x] = (1 + simulated_daily_returns.fillna(0)).cumprod()

simulated_cumulative_returns.head()

In [ ]:
simulated_price_df = pd.DataFrame({
    "Simulated AGG Prices": simulated_agg_prices,
    "Simulated SPY Prices": simulated_spy_prices
})

simulated_price_df.head()

In [ ]:
# Visualize the Simulation
plot_title = "Simulated Retirement Portfolio"
simulated_cumulative_returns.plot(legend=None, title=plot_title, figsize=(20,10))

In [ ]:
# Select the last row for the cumulative returns (cumulative returns at 30 years)
final_cumulative_returns = simulated_cumulative_returns.iloc[-1, :]

In [ ]:
# Display the 90% confidence interval for the ending returns
final_cumulative_returns.value_counts(bins=50) / len(final_cumulative_returns)
confidence_interval = final_cumulative_returns.quantile(q=[0.05, 0.95])
confidence_interval

In [ ]:
# Visualize the distribution of the ending returns
plt.figure();
final_cumulative_returns.plot(kind='hist', density=True, bins=50)
plt.axvline(confidence_interval.iloc[0], color='r')
plt.axvline(confidence_interval.iloc[1], color='r')

---

# Retirement Analysis

In this section, you will use the monte carlo model to answer the following retirement planning questions:

1. What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?
2. Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?
3. Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?
4. How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

### What are the expected cumulative returns at 30 years for the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given an initial investment of `$20,000`, what is the expected portfolio return in dollars at the 10th, 50th, and 90th percentiles?

In [ ]:
# YOUR CODE HERE

### Given the current projected annual income from the Plaid analysis, will a 4% withdraw rate from the retirement portfolio meet or exceed that value at the 10th percentile?

Note: This is effectively saying that 90% of the expected returns will be greater than the return at the 10th percentile, so this can help measure the uncertainty about having enough funds at retirement

In [ ]:
# YOUR CODE HERE

### How would a 50% increase in the initial investment amount affect the 4% retirement withdrawal?

In [ ]:
# YOUR CODE HERE

### Optional Challenge

In this section, you will calculate and plot the cumulative returns for the median and 90% confidence intervals. This plot shows the expected cumulative returns for any given day between the first day and the last day of investment. 

In [ ]:
# YOUR CODE HERE